In [36]:
import random
import numpy as np
from numpy.random import choice
import pandas as pd
import sys

In [37]:
totalSequence = 500
crossOver = 0.5
targetSequence = "31746025" # 15863724
positions = [x for x in range(8)] # for 8X8
maxFitness = 28

In [38]:
def calculate_fitness(srcSequence):
    sequence = [int(x) for x in srcSequence]
    horizontal_collisions = sum([sequence.count(queen)-1 for queen in sequence])/2
    n = len(sequence)
    
    left_diagonal = [0] * 2*n
    right_diagonal = [0] * 2*n
    
    for i in range(n):
        left_diagonal[i + sequence[i] - 1] += 1
        right_diagonal[len(sequence) - i + sequence[i] - 2] += 1
    
    diagonal_collisions = 0
    
    for i in range(2*n-1):
        counter = 0
        if left_diagonal[i] > 1:
            counter += left_diagonal[i]-1
        if right_diagonal[i] > 1:
            counter += right_diagonal[i]-1
        diagonal_collisions += counter / (n-abs(i-n+1))
    
    return int(maxFitness - (horizontal_collisions + diagonal_collisions)) 

In [39]:
def generate_initial_sequence():
    secureRandom = random.SystemRandom()
    initialPopulation = []
    sequenceScore = []
    for sequenceNum in range(totalSequence):
        randomSequence = []
        for sequence in range(len(targetSequence)):
            selectedPosition = str(secureRandom.choice(positions))
            randomSequence.append(selectedPosition)
        initialPopulation.append(randomSequence)
        sequenceScore.append(generate_sequence_score(randomSequence))
    return initialPopulation,sequenceScore

In [40]:
def generate_sequence_score(sequenceData):
    score = 0
    for sequence in range(len(targetSequence)):
        if(sequenceData[sequence] == targetSequence[sequence]):
            score = score + 1
    return score

In [41]:
def perform_crossover_mutation(parent1,parent2):
    secureRandom = random.SystemRandom()
    crossOverPoint = int(crossOver*len(targetSequence)) # default technique swap after 50% of sequence
    # cross over
    child1 = parent1[0:crossOverPoint]+parent2[crossOverPoint:]
    child2 = parent2[0:crossOverPoint]+parent1[crossOverPoint:]
    
    # Mutation
    child1[random.randint(0,len(targetSequence)-1)] = str(secureRandom.choice(positions))
    child2[random.randint(0,len(targetSequence)-1)] = str(secureRandom.choice(positions))
    
    #adding to initial sequence
    initialSequence.append(child1)
    initialSequence.append(child2)
    
    #calculating score
    sequenceScore.append(generate_sequence_score(child1))
    sequenceScore.append(generate_sequence_score(child2))
    
    #calculating probability
    probabilityDist.append(generate_sequence_score(child1)/len(targetSequence))
    probabilityDist.append(generate_sequence_score(child2)/len(targetSequence))
    

    return child1,child2
    
    

In [42]:
if __name__ == "__main__":
       
    srcseqScore = calculate_fitness(targetSequence)
    if srcseqScore != maxFitness:
        sys.exit('Not Valid Sequence. Skiiped Generation')
    else:
    
        initialSequence,sequenceScore = generate_initial_sequence()
        probabilityDist = []
        for sequenceNum in range(totalSequence):
            probabilityDist.append(sequenceScore[sequenceNum]/len(targetSequence))

        queenSequenceDF = pd.DataFrame({'QueenSequence':initialSequence,'SequenceScore':sequenceScore,'Probability':probabilityDist})
        queenSequenceDF = queenSequenceDF.sort_values(['Probability'],ascending=False)
        queenSequenceDF = queenSequenceDF.reset_index(drop=True)

        generationCount = 1000 # for total iteration to identify target sequence

        for loop in range(generationCount):
            parent1 = queenSequenceDF[0:1]["QueenSequence"].values[0]
            parent2 = queenSequenceDF[1:2]["QueenSequence"].values[0]

            # Incase of drawn data score matches with target
            if (generate_sequence_score(parent1)==len(targetSequence)):
                print("Valid Sequence Generated at iteration:",loop,'Sequence -',parent1)
                break
            elif(generate_sequence_score(parent2)==len(targetSequence)):
                print("Valid Sequence Generated at iteration:",loop,'Sequence -',parent2)
                break

            child1,child2 = perform_crossover_mutation(parent1,parent2)

            queenSequenceDF = pd.DataFrame({'QueenSequence':initialSequence,'SequenceScore':sequenceScore,'Probability':probabilityDist})
            queenSequenceDF = queenSequenceDF.sort_values(['Probability'],ascending=False)
            queenSequenceDF = queenSequenceDF.reset_index(drop=True)



            print('Generation ',loop,' ',' Average Sequence Score ',queenSequenceDF["SequenceScore"].mean(),' ',
                  ''.join(str(elem) for elem in child1),' ',generate_sequence_score(child1),
                  ''.join(str(elem) for elem in child2),generate_sequence_score(child2)
                 )

Generation  0    Average Sequence Score  1.0079681274900398   71745422   4 31406222 4
Generation  1    Average Sequence Score  1.0178571428571428   71446222   4 51406222 3
Generation  2    Average Sequence Score  1.0296442687747036   71776126   4 07746222 4
Generation  3    Average Sequence Score  1.0413385826771653   71746226   5 07736222 3
Generation  4    Average Sequence Score  1.0568627450980392   71746212   4 71746225 6
Generation  5    Average Sequence Score  1.0703125   71740226   4 71746223 5
Generation  6    Average Sequence Score  1.083657587548638   71741223   4 71746223 5
Generation  7    Average Sequence Score  1.0968992248062015   71546223   4 71346225 5
Generation  8    Average Sequence Score  1.11003861003861   71766225   5 71341225 4
Generation  9    Average Sequence Score  1.123076923076923   71726225   5 71767225 4
Generation  10    Average Sequence Score  1.1379310344827587   71746236   4 21746225 6
Generation  11    Average Sequence Score  1.1583969465648856   217